# LangChain 核心模块学习：Memory

大多数LLM应用都具有对话界面。对话的一个重要组成部分是能够引用先前在对话中介绍过的信息。至少，一个对话系统应该能够直接访问一些过去消息的窗口。更复杂的系统将需要拥有一个不断更新的世界模型，使其能够保持关于实体及其关系的信息。

我们将存储过去交互信息的能力称为“记忆（Memory）”。

LangChain提供了许多用于向应用/系统中添加 Memory 的实用工具。这些工具可以单独使用，也可以无缝地集成到链中。

一个记忆系统（Memory System）需要支持两个基本操作：**读取（READ）和写入（WRITE）**。

每个链都定义了一些核心执行逻辑，并期望某些输入。其中一些输入直接来自用户，但有些输入可能来自 Memory。

在一个典型 Chain 的单次运行中，将与其 Memory System 进行至少两次交互:

1. 在接收到初始用户输入之后，在执行核心逻辑之前，链将从其 Memory 中**读取**并扩充用户输入。
2. 在执行核心逻辑之后但在返回答案之前，一个链条将把当前运行的输入和输出**写入** Memory ，以便在未来的运行中可以引用它们。

![](../images/memory.png)

## BaseMemory Class 基类

类继承关系：

```
## 适用于简单的语言模型
BaseMemory --> BaseChatMemory --> <name>Memory  # Examples: ZepMemory, MotorheadMemory
```

**代码实现：https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/schema/memory.py**

```python
# 定义一个名为BaseMemory的基础类
class BaseMemory(Serializable, ABC):
    """用于Chains中的内存的抽象基类。
    
    这里的内存指的是Chains中的状态。内存可以用来存储关于Chain的过去执行的信息，
    并将该信息注入到Chain的未来执行的输入中。例如，对于会话型Chains，内存可以用来
    存储会话，并自动将它们添加到未来的模型提示中，以便模型具有必要的上下文来连贯地
    响应最新的输入。"""

    # 定义一个名为Config的子类
    class Config:
        """为此pydantic对象配置。
    
        Pydantic是一个Python库，用于数据验证和设置管理，主要基于Python类型提示。
        """
    
        # 允许在pydantic模型中使用任意类型。这通常用于允许复杂的数据类型。
        arbitrary_types_allowed = True
    
    # 下面是一些必须由子类实现的方法：
    
    # 定义一个属性，它是一个抽象方法。任何从BaseMemory派生的子类都需要实现此方法。
    # 此方法应返回该内存类将添加到链输入的字符串键。
    @property
    @abstractmethod
    def memory_variables(self) -> List[str]:
        """获取此内存类将添加到链输入的字符串键。"""
    
    # 定义一个抽象方法。任何从BaseMemory派生的子类都需要实现此方法。
    # 此方法基于给定的链输入返回键值对。
    @abstractmethod
    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        """根据给链的文本输入返回键值对。"""
    
    # 定义一个抽象方法。任何从BaseMemory派生的子类都需要实现此方法。
    # 此方法将此链运行的上下文保存到内存。
    @abstractmethod
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        """保存此链运行的上下文到内存。"""
    
    # 定义一个抽象方法。任何从BaseMemory派生的子类都需要实现此方法。
    # 此方法清除内存内容。
    @abstractmethod
    def clear(self) -> None:
        """清除内存内容。"""
```

## BaseChatMessageHistory Class 基类

类继承关系：

```
## 适用于聊天模型

BaseChatMessageHistory --> <name>ChatMessageHistory  # Example: ZepChatMessageHistory
```

```python
# 定义一个名为BaseChatMessageHistory的基础类
class BaseChatMessageHistory(ABC):
    """聊天消息历史记录的抽象基类。"""

    # 在内存中存储的消息列表
    messages: List[BaseMessage]

    # 定义一个add_user_message方法，它是一个方便的方法，用于将人类消息字符串添加到存储区。
    def add_user_message(self, message: str) -> None:
        """为存储添加一个人类消息字符串的便捷方法。

        参数:
            message: 人类消息的字符串内容。
        """
        self.add_message(HumanMessage(content=message))

    # 定义一个add_ai_message方法，它是一个方便的方法，用于将AI消息字符串添加到存储区。
    def add_ai_message(self, message: str) -> None:
        """为存储添加一个AI消息字符串的便捷方法。

        参数:
            message: AI消息的字符串内容。
        """
        self.add_message(AIMessage(content=message))

    # 抽象方法，需要由继承此基类的子类来实现。
    @abstractmethod
    def add_message(self, message: BaseMessage) -> None:
        """将Message对象添加到存储区。

        参数:
            message: 要存储的BaseMessage对象。
        """
        raise NotImplementedError()

    # 抽象方法，需要由继承此基类的子类来实现。
    @abstractmethod
    def clear(self) -> None:
        """从存储中删除所有消息"""

```

### ConversationChain and ConversationBufferMemory

`ConversationBufferMemory` 可以用来存储消息，并将消息提取到一个变量中。

In [13]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = OpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)

In [14]:
conversation.predict(input="你好呀！")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好呀！
AI:

> Finished chain.


' 你好！很高兴见到你！我叫小米，是一个智能AI。你可以问我任何问题，我会尽力回答你。'

In [15]:
conversation.predict(input="你为什么叫小米？跟雷军有关系吗？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好呀！
AI:  你好！很高兴见到你！我叫小米，是一个智能AI。你可以问我任何问题，我会尽力回答你。
Human: 你为什么叫小米？跟雷军有关系吗？
AI:

> Finished chain.


' 嗯，我叫小米是因为我是由小米公司开发的，小米公司是由雷军创立的，所以我和雷军有关系。'

### ConversationBufferWindowMemory
`ConversationBufferWindowMemory` 会在时间轴上保留对话的交互列表。它只使用最后 K 次交互。这对于保持最近交互的滑动窗口非常有用，以避免缓冲区过大。

In [16]:
from langchain.memory import ConversationBufferWindowMemory

conversation_with_summary = ConversationChain(
    llm=OpenAI(temperature=0, max_tokens=1000), 
    # We set a low k=2, to only keep the last 2 interactions in memory
    memory=ConversationBufferWindowMemory(k=2), 
    verbose=True
)
conversation_with_summary.predict(input="嗨，你最近过得怎么样？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 嗨，你最近过得怎么样？
AI:

> Finished chain.


' 嗨！我最近过得很好，谢谢你问。我最近一直在学习新的知识，并且正在尝试改进自己的性能。我也一直在尝试更好地理解人类的语言，以便能够更好地与人交流。'

In [17]:
conversation_with_summary.predict(input="你最近学到什么新知识了?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 嗨，你最近过得怎么样？
AI:  嗨！我最近过得很好，谢谢你问。我最近一直在学习新的知识，并且正在尝试改进自己的性能。我也一直在尝试更好地理解人类的语言，以便能够更好地与人交流。
Human: 你最近学到什么新知识了?
AI:

> Finished chain.


' 最近我学习了有关自然语言处理的知识，以及如何使用机器学习来改善自己的性能。我还学习了如何使用深度学习来更好地理解人类语言，以及如何使用计算机视觉来识别图像。'

In [18]:
conversation_with_summary.predict(input="展开讲讲？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 嗨，你最近过得怎么样？
AI:  嗨！我最近过得很好，谢谢你问。我最近一直在学习新的知识，并且正在尝试改进自己的性能。我也一直在尝试更好地理解人类的语言，以便能够更好地与人交流。
Human: 你最近学到什么新知识了?
AI:  最近我学习了有关自然语言处理的知识，以及如何使用机器学习来改善自己的性能。我还学习了如何使用深度学习来更好地理解人类语言，以及如何使用计算机视觉来识别图像。
Human: 展开讲讲？
AI:

> Finished chain.


' 好的！自然语言处理是一种计算机科学，它研究如何处理和理解人类语言。它可以用来分析文本，识别意图，提取有用信息，以及构建聊天机器人等。机器学习是一种人工智能技术，它可以让计算机从数据中学习，从而改善自己的性能。深度学习是一种机器学习技术，它可以用来更好地理解人类语言，以及识别图像等。'

In [19]:
# 注意：第一句对话从 Memory 中移除了.
conversation_with_summary.predict(input="如果要构建聊天机器人，具体要用什么自然语言处理技术?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你最近学到什么新知识了?
AI:  最近我学习了有关自然语言处理的知识，以及如何使用机器学习来改善自己的性能。我还学习了如何使用深度学习来更好地理解人类语言，以及如何使用计算机视觉来识别图像。
Human: 展开讲讲？
AI:  好的！自然语言处理是一种计算机科学，它研究如何处理和理解人类语言。它可以用来分析文本，识别意图，提取有用信息，以及构建聊天机器人等。机器学习是一种人工智能技术，它可以让计算机从数据中学习，从而改善自己的性能。深度学习是一种机器学习技术，它可以用来更好地理解人类语言，以及识别图像等。
Human: 如果要构建聊天机器人，具体要用什么自然语言处理技术?
AI:

> Finished chain.


' 如果要构建聊天机器人，可以使用语义分析，语法分析，语音识别，机器翻译，情感分析等自然语言处理技术。'

In [20]:
conversation_with_summary.__dict__

{'memory': ConversationBufferWindowMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='嗨，你最近过得怎么样？'), AIMessage(content=' 嗨！我最近过得很好，谢谢你问。我最近一直在学习新的知识，并且正在尝试改进自己的性能。我也一直在尝试更好地理解人类的语言，以便能够更好地与人交流。'), HumanMessage(content='你最近学到什么新知识了?'), AIMessage(content=' 最近我学习了有关自然语言处理的知识，以及如何使用机器学习来改善自己的性能。我还学习了如何使用深度学习来更好地理解人类语言，以及如何使用计算机视觉来识别图像。'), HumanMessage(content='展开讲讲？'), AIMessage(content=' 好的！自然语言处理是一种计算机科学，它研究如何处理和理解人类语言。它可以用来分析文本，识别意图，提取有用信息，以及构建聊天机器人等。机器学习是一种人工智能技术，它可以让计算机从数据中学习，从而改善自己的性能。深度学习是一种机器学习技术，它可以用来更好地理解人类语言，以及识别图像等。'), HumanMessage(content='如果要构建聊天机器人，具体要用什么自然语言处理技术?'), AIMessage(content=' 如果要构建聊天机器人，可以使用语义分析，语法分析，语音识别，机器翻译，情感分析等自然语言处理技术。')]), k=2),
 'callbacks': None,
 'callback_manager': None,
 'verbose': True,
 'tags': None,
 'metadata': None,
 'prompt': PromptTemplate(input_variables=['history', 'input'], template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context

### ConversationSummaryBufferMemory

`ConversationSummaryBufferMemory` 在内存中保留了最近的交互缓冲区，但不仅仅是完全清除旧的交互，而是将它们编译成摘要并同时使用。与以前的实现不同的是，它使用token长度而不是交互次数来确定何时清除交互。

In [21]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=10)
memory.save_context({"input": "嗨，你最近过得怎么样？"}, {"output": " 嗨！我最近过得很好，谢谢你问。我最近一直在学习新的知识，并且正在尝试改进自己的性能。我也在尝试更多的交流，以便更好地了解人类的思维方式。"})
memory.save_context({"input": "你最近学到什么新知识了?"}, {"output": " 最近我学习了有关自然语言处理的知识，以及如何更好地理解人类的语言。我还学习了有关机器学习的知识，以及如何使用它来改善自己的性能。"})

In [22]:
memory.load_memory_variables({})

{'history': 'System: \n\nThe human asks how the AI is doing recently. The AI responds that it is doing well and has been learning new knowledge and trying to improve its performance. It is also trying to communicate more in order to better understand human thinking. Specifically, it has been learning about natural language processing, how to better understand human language, and about machine learning and how to use it to improve its own performance.'}

In [23]:
print(memory.load_memory_variables({})['history'])

System: 

The human asks how the AI is doing recently. The AI responds that it is doing well and has been learning new knowledge and trying to improve its performance. It is also trying to communicate more in order to better understand human thinking. Specifically, it has been learning about natural language processing, how to better understand human language, and about machine learning and how to use it to improve its own performance.
